In [1]:
from pocketminer_stats import Results, save_to_csv
import csv 
from typing import Dict
import os
import numpy as np


PATH = '/home/vit/Projects/cryptobench/data/H-prediction-evaluation/ahoj-v2/p2rank/'
INPUT_PATH = f'{PATH}/p2rank-predictions'
OUTPUT_PATH = f'{PATH}/p2rank-stats'
# how is the multichain dealt with?

In [2]:
def sigmoid(z):
    return 1/(1 + np.exp(-z))

def read_predictions(binary_classification_path, raw_score_path) -> Dict[str, Results]:
    result = {}
    
    for file in os.listdir(binary_classification_path):
        chains_1 = set() # chains from binary_classification_path file
        chains_2 = set() # chains from raw_score_path file
        predictions = []
        raw_scores = []
        actual_values = []

        with open(f'{binary_classification_path}/{file}', 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            next(csvfile)
            for row in reader:
                
                assert row[0].strip() == row[1].strip(), row
                chain = row[0].strip().split('_')[0]
                chains_1.add(chain)

                actual_val = int(row[2])
                prediction = int(row[3])

                predictions.append(prediction)
                actual_values.append(actual_val)
        
        with open(f'{raw_score_path}/{file}', 'r') as csvfile:
            reader = csv.reader(csvfile, delimiter=',')
            next(reader)
            for row in reader:
                chains_2.add(row[0])
                score = float(row[3])
                raw_scores.append(score)

        assert len(raw_scores) == len(predictions)
        assert chains_1 == chains_2

        pdb_id = file.split('.')[0]
        chain_id = '-'.join(chains_1)
        
        predictions = np.array(predictions)
        raw_scores = np.array(raw_scores)

        result[f'{pdb_id}{chain_id}'] = Results(
            actual_values, predictions, sigmoid(raw_scores), f'{pdb_id}{chain_id}')

    return result




In [3]:
apo_results = read_predictions(f'{INPUT_PATH}/eval_predict_single-pair-test_apo/cases', f'{INPUT_PATH}/predict_single-pair-test_apo')
holo_results = read_predictions(f'{INPUT_PATH}/eval_predict_single-pair-test_holo/cases', f'{INPUT_PATH}/predict_single-pair-test_holo')


save_to_csv(apo_results, f'{OUTPUT_PATH}/p2rank-test-subset-apo.csv')   
save_to_csv(holo_results, f'{OUTPUT_PATH}/p2rank-test-subset-holo.csv')   



In [25]:
max_score = 0
min_score = 1
for i, val in apo_results.items():
    for i in range(len(val.actual_values)):
        if val.predictions[i] == 1:
            min_score = min(min_score, val.predictions_for_auc[i])
        else:
            max_score = max(max_score, val.predictions_for_auc[i])

print(max_score)
print(min_score)


max_score = 0
min_score = 1
for i, val in holo_results.items():
    max_score = 0
    min_score = 1

    for i in range(len(val.actual_values)):
        if val.predictions[i] == 1:
            min_score = min(min_score, val.predictions_for_auc[i])
        else:
            max_score = max(max_score, val.predictions_for_auc[i])
        #assert val.predictions_for_auc[i] < 0.53 or val.predictions_for_auc[i] > 0.73, val.predictions_for_auc[i]

    print(max_score, min_score)


0.730901260011949
0.522534723453772
0.7227011531939176 0.5599107872632886
0.7294038364446794 0.5550015234449647
0.6992859647672691 0.5471841823215133
0.6984652160025387 0.609854343522874
0.6570780638031497 0.5664790559676278
0.7181290608919957 0.5907338742589551
0.7156529818288933 0.5323796216453274
0.7284553992819265 0.600392309828242
0.7208939053398985 0.5338979215658985
0.7290681700244377 0.5670929049654543
0.622905734254928 0.5415539317112741
0.6899958716908805 0.5587276779576763
0.7246009594254244 0.6107343333232297
0.7255179606769018 0.535117086185012
0.6565370765609826 0.5676329218314142
0.6659446808847085 0.5361120122638564
0.6363137004961562 0.5583084994073467
0.7015312062984412 0.5657667355872839
0.6386708721142788 0.5714084719348067
0.7035165494777081 0.7667589486719418
0.7309799265884654 0.5521595465928841
0.7154290855686125 0.535117086185012
0.707340014518073 0.5616595520036753
0.7291274242840583 0.5616349320419256
0.7078365914591789 0.5628163457038591
0.6936204040838114 0